In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
#read in data
def readdata(fname):
    hyperspec = pd.read_csv(fname, delimiter = ',', header=0, dtype = np.float64)
    return hyperspec

In [3]:
#converts wavelength to energy, averages reflectance data, performs Kulbelka-Munk transformation, plots data for reference
def dataconvert(hsdata):
    Energy = (1/(hsdata[hsdata.keys()[0]]/2.998e17))*4.1357e-15
    Energy = Energy[::-1]
    avg = hsdata[hsdata.keys()[1:]].mean(axis=1)
    Refavg = avg[::-1]
    FRavg = (1-Refavg)**2/(2*Refavg)
    indirectavg = (FRavg*Energy)**0.5
    directavg = (FRavg*Energy)**2
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(Energy,directavg,'o')
    ax1.set_title('Direct Bandgap')
    ax2.plot(Energy,indirectavg,'o')
    ax2.set_title('Indirect Bandgap')
    return [Energy, FRavg, directavg, indirectavg]

In [4]:
#Tauc plot fitting to determine direct/indirect band gap based on the limits given in terms of energy
def manualbandgap(hsdata, lim1, lim2):
    x = hsdata[0]
    diry = hsdata[2]
    indiry = hsdata[3]
    LinearEnergy = []
    LinearDirect = []
    LinearIndirect = []
    for k in range(len(x)):
        if lim1 < x[k] < lim2:
            LinearEnergy.append(x[k])
            LinearDirect.append(diry[k])
    directcoef = np.polyfit(LinearEnergy,LinearDirect,deg=1)
    directpoly1d_fn = np.poly1d(directcoef)
    LinearIndirect = []
    for k in range(len(x)):
        if lim1 < x[k] < lim2:
            LinearIndirect.append(indiry[k])
    indirectcoef = np.polyfit(LinearEnergy,LinearIndirect,deg=1)
    indirectpoly1d_fn = np.poly1d(indirectcoef)
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(x, diry, 'yo', x, directpoly1d_fn(x), '--k')
    ax1.set_title('Direct Bandgap')
    ax1.set_xlim(min(x),max(x))
    ax1.set_ylim(0,max(diry))
    ax2.plot(x,indiry, 'yo', x, indirectpoly1d_fn(x), '--k')
    ax2.set_title('Indirect Bandgap')
    ax2.set_xlim(min(x),max(x))
    ax2.set_ylim(0,max(indiry))
    print('Direct Bandgap is', directcoef[1]/np.negative(directcoef[0]), 'Indirect Bangap is', indirectcoef[1]/np.negative(indirectcoef[0]))
    return [directcoef[0], directcoef[1], indirectcoef[0], indirectcoef[1]]

In [5]:
#does above 3 functions all together
def automatedbandgap(fname, lim1, lim2):
    hyperspec = pd.read_csv(fname, delimiter = ',', header=0, dtype = np.float64)
    hsdata = dataconvert(hyperspec)
    return manualbandgap(hsdata, lim1, lim2), hsdata

In [6]:
#automatically performs data conversion and Tauc plot fitting 
#for series of data sets using one set of limits for the linear region
#exports the data for plotting outside of python
def autoHype(path,lim1,lim2):
    spectra = {}
    for file in Path(path).glob('*.csv'):
        spectra.update({str(file):[]})
    for s in spectra:
        spectra[s].append(automatedbandgap(s,lim1,lim2))
    for s in spectra:
        line = spectra[s][0][1][0]*spectra[s][0][0][0]+spectra[s][0][0][1]
        np.savetxt(s+'-Linear-Fit',line)
        np.savetxt(s+'-Energy',spectra[s][0][1][0]) #Saves Energy
        np.savetxt(s+'-Direct',spectra[s][0][1][2]) #Saves Direct Bandgap Refl
        np.savetxt(s+'-Indirect',spectra[s][0][1][2]) #Save Indirect Bandgap Refl
    plt.figure()
    for s in spectra:
        plt.scatter(spectra[s][0][0][1]/(-spectra[s][0][0][0]),spectra[s][0][0][2]/(-spectra[s][0][0][3]),label=s)
    plt.xlabel('Direct')
    plt.ylabel('Indirect')
    plt.legend()
    return spectra

In [7]:
spectra = autoHype('.',2.725,2.9)

In [8]:
#find the intersection between two lines
def get_intersection(m1,b1,m2,b2):
    A = np.array([[-m1, 1], [-m2, 1]])
    b = np.array([[b1], [b2]])
    # you have to solve linear System AX = b where X = [x y]'
    X = np.linalg.pinv(A) @ b
    x, y = np.round(np.squeeze(X), 4)
    return x, y # returns point of intersection (x,y) with 4 decimal precision

In [9]:
#this corrects a data set if there is absorption outside of the linear region to get the "true" band gap
def manualbandgapcorrected(hsdata, lim1, lim2, lim3, lim4):
    x = hsdata[0]
    diry = hsdata[2]
    indiry = hsdata[3]
    LinearEnergy = []
    LinearDirect = []
    LinearIndirect = []
    LinearEnergy2 = []
    LinearDirect2 = []
    LinearIndirect2 = []
    for k in range(len(x)):
        if lim1 < x[k] < lim2:
            LinearEnergy.append(x[k])
            LinearDirect.append(diry[k])
    directcoef = np.polyfit(LinearEnergy,LinearDirect,deg=1)
    directpoly1d_fn = np.poly1d(directcoef)
    LinearIndirect = []
    for k in range(len(x)):
        if lim1 < x[k] < lim2:
            LinearIndirect.append(indiry[k])
    indirectcoef = np.polyfit(LinearEnergy,LinearIndirect,deg=1)
    indirectpoly1d_fn = np.poly1d(indirectcoef)
    for k in range(len(x)):
        if lim3 < x[k] < lim4:
            LinearEnergy2.append(x[k])
            LinearDirect2.append(diry[k])
    directcoef2 = np.polyfit(LinearEnergy2,LinearDirect2,deg=1)
    directpoly1d_fn2 = np.poly1d(directcoef2)
    LinearIndirect = []
    for k in range(len(x)):
        if lim3 < x[k] < lim4:
            LinearIndirect2.append(indiry[k])
    indirectcoef2 = np.polyfit(LinearEnergy2,LinearIndirect2,deg=1)
    indirectpoly1d_fn2 = np.poly1d(indirectcoef2)
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(x, diry, 'yo', x, directpoly1d_fn(x), '--k', x, directpoly1d_fn2(x),'--k')
    ax1.set_title('Direct Bandgap')
    ax1.set_xlim(min(x),max(x))
    ax1.set_ylim(0,max(diry))
    ax2.plot(x,indiry, 'yo', x, indirectpoly1d_fn(x), '--k', x, indirectpoly1d_fn2(x),'--k')
    ax2.set_title('Indirect Bandgap')
    ax2.set_xlim(min(x),max(x))
    ax2.set_ylim(0,max(indiry))
    dbg=get_intersection(directcoef[0],directcoef[1],directcoef2[0],directcoef2[1])
    ibg=get_intersection(indirectcoef[0],indirectcoef[1],indirectcoef2[0],indirectcoef2[1])
    print('Direct Bandgap is', dbg[0], 'Indirect Bangap is', ibg[0])
    np.savetxt('Corrected-linearfit1',directpoly1d_fn(x))
    np.savetxt('Corrected-linearfit2',directpoly1d_fn2(x))
    return [directcoef[1]/np.negative(directcoef[0]), indirectcoef[1]/np.negative(indirectcoef[0]),dbg[0],ibg[0]]

In [10]:
manualbandgapcorrected(dataconvert(readdata('BWO-FZ-cut1.csv')),2.75,2.9,2,2.5)

In [14]:
%matplotlib notebook
plt.style.use(['science','notebook'])
#load the manually corrected fit
fit1, fit2 = np.loadtxt('Corrected-linearfit1'),np.loadtxt('Corrected-linearfit2')
#plotting
fig, axes = plt.subplots(2, 2)
keynames = list(spectra.keys())
axes[0,0].plot(spectra[keynames[0]][0][1][0], spectra[keynames[0]][0][1][2], 'o', spectra[keynames[0]][0][1][0],(spectra[keynames[0]][0][1][0]*spectra[keynames[0]][0][0][0]+spectra[keynames[0]][0][0][1]), '--k')
axes[0,1].plot(spectra[keynames[3]][0][1][0], spectra[keynames[3]][0][1][2], 'o', spectra[keynames[3]][0][1][0],(spectra[keynames[3]][0][1][0]*spectra[keynames[3]][0][0][0]+spectra[keynames[3]][0][0][1]), '--k')
axes[1,0].plot(spectra[keynames[2]][0][1][0], spectra[keynames[2]][0][1][2], 'o', spectra[keynames[2]][0][1][0],(spectra[keynames[2]][0][1][0]*spectra[keynames[2]][0][0][0]+spectra[keynames[2]][0][0][1]), '--k',spectra[keynames[2]][0][1][0],fit2,'--k')
axes[1,1].plot(spectra[keynames[4]][0][1][0], spectra[keynames[4]][0][1][2], 'o', spectra[keynames[4]][0][1][0],(spectra[keynames[4]][0][1][0]*spectra[keynames[4]][0][0][0]+spectra[keynames[4]][0][0][1]), '--k')
#formatting axes
axes[0,0].set_xlim(1.4,3.1)
axes[0,0].set_ylim(0,80)
axes[0,0].text(1.45,67,'a) Bi$_{1.860}$W$_{0.924}$O$_{5.562}$ flux \nE$_g$~2.5 eV',fontsize='large', va = 'center')
axes[0,1].set_xlim(1.4,3.1)
axes[0,1].set_ylim(0,60)
axes[0,1].text(1.45,50.25,'b) Bi$_2$W$_{0.99}$Fe$_{0.01}$O$_6$ Fe-doped flux \nE$_g$~2.6 eV',fontsize='large', va = 'center')
axes[1,0].set_xlim(1.4,3.1)
axes[1,0].set_ylim(0,20)
axes[1,0].text(1.45,16.75,'c) Bi$_{1.887}$W$_{0.934}$O$_{5.384}$ FZ \nE$_{g-corr}$~2.7 eV',fontsize='large', va = 'center')
axes[1,0].set_xlim(1.4,3.1)
axes[1,0].set_ylim(0,20)
axes[1,1].text(1.45,16.75,'d) Bi$_{1.887}$W$_{0.934}$O$_{5.622}$ FZ \nE$_g$~2.7 eV',fontsize='large', va = 'center')
axes[1,1].set_xlim(1.4,3.1)
axes[1,1].set_ylim(0,20)
fig.supxlabel('Energy (eV)',fontsize = 'xx-large')
fig.supylabel('[F(R)$*$h$\\nu$]$^2$ (a.u.)',fontsize = 'xx-large')
fig.savefig('Hyperspectral-updated-20230830', dpi=300)

In [23]:
keynames

['BWO-flux.csv',
 'BWO-FZ-cut2.csv',
 'BWO-FZ-cut1.csv',
 'BWO-Fedoped-flux.csv',
 'BWO-ox-FZ.csv']

In [16]:
spectra[0]

{'BWO-flux.csv': [([181.41943046796962,
    -447.83201567518046,
    2.117500526503837,
    -3.1517695300418365],
   [370    1.238199
    369    1.240208
    368    1.242228
    367    1.244256
    366    1.246291
             ...   
    4      3.054064
    3      3.066353
    2      3.078733
    1      3.091222
    0      3.103812
    Name: wavelength, Length: 371, dtype: float64,
    370    0.000166
    369    0.000005
    368    0.000036
    367    0.000303
    366    0.000317
             ...   
    4      1.334506
    3      1.130524
    2      1.026414
    1      0.824460
    0      0.786770
    Length: 371, dtype: float64,
    370    4.204838e-08
    369    4.077441e-11
    368    1.973328e-09
    367    1.423037e-07
    366    1.565608e-07
               ...     
    4      1.661105e+01
    3      1.201721e+01
    2      9.985947e+00
    1      6.495306e+00
    0      5.963304e+00
    Length: 371, dtype: float64,
    370    0.014320
    369    0.002527
    368    0.006665
    3